In [29]:
import os
import re
from pathlib import Path

import numpy as np
import pandas as pd
import geopandas as gpd
from tqdm import tqdm
tqdm.pandas()

from shapely.geometry import MultiPolygon, Polygon
from shapely.validation import make_valid

In [30]:
from constants import CENSUS_25

BUFFER_DIST = 3000

In [31]:
# Load GTA municipalities boundary
gta = gpd.read_file("../data/geo/regions/TMUN.gpkg")

# Ensure geometry validity
gta["geometry"] = gta["geometry"].apply(make_valid)

# Reproject
CRS_METERS = "EPSG:32617"
gta_proj = gta.to_crs(CRS_METERS)

# Dissolve for boundary use
gta_boundary = gta_proj.dissolve().geometry.values[0]  # shapely Polygon/MultiPolygon

# Create a 5 km buffer around GTA
gta_buffer = gta_boundary.buffer(BUFFER_DIST)

# Grid resolution
cell_size = 1000  # meters

# Create bounding box for grid
minx, miny, maxx, maxy = gta_buffer.bounds  # use buffer bounds

# Grid coordinates
x_coords = np.arange(minx, maxx + cell_size, cell_size)
y_coords = np.arange(miny, maxy + cell_size, cell_size)

# Create squares only inside GTA buffer
grid_polys = []
for x in x_coords:
    for y in y_coords:
        poly = Polygon([
            (x, y),
            (x + cell_size, y),
            (x + cell_size, y + cell_size),
            (x, y + cell_size)
        ])
        if poly.intersects(gta_buffer):
            grid_polys.append(poly)

grid = gpd.GeoDataFrame({"geometry": grid_polys}, crs=CRS_METERS)

# Save as GeoPackage
OUT_PATH = "../data/geo/regions/TMUN_squares_fullgrid.gpkg"
grid.to_file(OUT_PATH, driver="GPKG")

grid.head()

,geometry
0,"POLYGON ((586570.163 4833163.341, 587570.163 4..."
1,"POLYGON ((586570.163 4834163.341, 587570.163 4..."
2,"POLYGON ((586570.163 4835163.341, 587570.163 4..."
3,"POLYGON ((586570.163 4836163.341, 587570.163 4..."
4,"POLYGON ((586570.163 4837163.341, 587570.163 4..."


In [32]:
SQUARES_PATH = "../data/geo/regions/TMUN_squares_fullgrid.gpkg"
OUT_PATTERN = "../data/language/{year}/num_speakers_squares_{year}.gpkg"

def _ensure_out_dir(path_str):
    Path(path_str).parent.mkdir(parents=True, exist_ok=True)

# Load grid
squares = gpd.read_file(SQUARES_PATH).to_crs(CRS_METERS).reset_index(drop=True)

# Spatial index for intersections
squares_sindex = squares.sindex

# Main loop over census years
for year in CENSUS_25:
    print(f"\nInterpolating year {year}...")

    # Load tract-level counts
    tract_path = f"../data/language/{year}/num_speakers_tmun_{year}.gpkg"
    tracts = gpd.read_file(tract_path).to_crs(CRS_METERS)

    # Columns to interpolate
    num_cols = [c for c in tracts.columns if c.startswith("num_") and c != "geometry"]
    if "num_tot" not in num_cols and "num_tot" in tracts.columns:
        num_cols.append("num_tot")

    # Prepare result GeoDataFrame
    result = squares.copy()
    for c in num_cols:
        result[c] = 0.0

    tracts[num_cols] = tracts[num_cols].astype(float)

    # Distribute tract counts to intersecting squares
    for idx, tract in tqdm(tracts.iterrows(), total=len(tracts), desc=f"year {year} tracts"):
        tract_geom = tract.geometry
        if tract_geom is None or tract_geom.is_empty:
            continue
        tract_area = tract_geom.area
        if tract_area == 0 or np.isnan(tract_area):
            continue

        # candidate square indices whose bounding boxes intersect tract bbox
        possible_idx = list(squares_sindex.intersection(tract_geom.bounds))
        if not possible_idx:
            continue

        candidates = result.iloc[possible_idx]
        intersections = candidates.geometry.intersection(tract_geom)
        inter_areas = intersections.area
        mask = inter_areas > 0
        if not mask.any():
            continue

        recv_idx = candidates.index[mask]
        ratios = inter_areas[mask] / tract_area

        for col in num_cols:
            tract_value = tract[col] if pd.notnull(tract[col]) else 0.0
            if tract_value == 0:
                continue
            apportioned = ratios * float(tract_value)
            result.loc[recv_idx, col] += apportioned.values

    # Ensure no negative values
    for c in num_cols:
        result[c] = result[c].clip(lower=0.0)

    # Save interpolated squares
    out_path = OUT_PATTERN.format(year=year)
    _ensure_out_dir(out_path)
    result.to_file(out_path, driver="GPKG")
    print(f"Saved interpolated squares for {year} -> {out_path}")

print("\nAll years complete.")


Interpolating year 1971...


year 1971 tracts: 100%|██████████| 422/422 [00:04<00:00, 100.33it/s]


Saved interpolated squares for 1971 -> ../data/language/1971/num_speakers_squares_1971.gpkg

Interpolating year 1996...


year 1996 tracts: 100%|██████████| 713/713 [00:09<00:00, 75.46it/s]


Saved interpolated squares for 1996 -> ../data/language/1996/num_speakers_squares_1996.gpkg

Interpolating year 2021...


year 2021 tracts: 100%|██████████| 1049/1049 [00:18<00:00, 55.77it/s]


Saved interpolated squares for 2021 -> ../data/language/2021/num_speakers_squares_2021.gpkg

All years complete.


In [33]:
IN_PATTERN = "../data/language/{year}/num_speakers_squares_{year}.gpkg"
PCT_PATTERN = "../data/language/{year}/pct_speakers_squares_{year}.gpkg"
CENT_NUM_PATTERN = "../data/language/{year}/num_speakers_centroid_{year}.gpkg"
CENT_PCT_PATTERN = "../data/language/{year}/pct_speakers_centroid_{year}.gpkg"

for year in CENSUS_25:
    print(f"Processing {year}…")

    # Load NUM dataset
    num_path = IN_PATTERN.format(year=year)
    gdf_num = gpd.read_file(num_path)

    lang_cols = [c for c in gdf_num.columns if c.startswith("num_") and c != "num_tot"]

    # Compute percentages
    gdf_pct = gdf_num.copy()
    EPS = 1e-9
    safe_tot = gdf_pct["num_tot"].replace(0, EPS)
    for col in lang_cols:
        lang = col.replace("num_", "")
        gdf_pct[f"pct_{lang}"] = (gdf_pct[col] / safe_tot * 100).round(2)

    zero_mask = gdf_pct["num_tot"] == 0
    for col in gdf_pct.columns:
        if col.startswith("pct_"):
            gdf_pct.loc[zero_mask, col] = 0

    gdf_pct["num_tot"] = gdf_pct["num_tot"].round(2)
    gdf_pct = gdf_pct.drop(columns=lang_cols)

    # Save pct file
    pct_path = PCT_PATTERN.format(year=year)
    gdf_pct.to_file(pct_path, driver="GPKG")

    # Round NUM values
    for col in lang_cols + ["num_tot"]:
        gdf_num[col] = gdf_num[col].round(2)
    gdf_num.to_file(num_path, driver="GPKG")

    # Centroid versions
    gdf_num_cent = gdf_num.copy()
    gdf_num_cent["geometry"] = gdf_num_cent.centroid
    gdf_num_cent.to_file(CENT_NUM_PATTERN.format(year=year), driver="GPKG")

    gdf_pct_cent = gdf_pct.copy()
    gdf_pct_cent["geometry"] = gdf_pct_cent.centroid
    gdf_pct_cent.to_file(CENT_PCT_PATTERN.format(year=year), driver="GPKG")

    print(f"✓ Finished {year}")

Processing 1971…
✓ Finished 1971
Processing 1996…
✓ Finished 1996
Processing 2021…
✓ Finished 2021


In [35]:
from shapely.geometry import Point
import json

# Load GTA boundary in WGS84
gta = gpd.read_file("../data/geo/regions/TMUN.gpkg").to_crs("EPSG:4326")
gta_boundary = gta.dissolve().geometry.values[0]  # Polygon/MultiPolygon

# --- NEW: Create a 5 km *expanded* boundary for filtering ---
CRS_METERS = "EPSG:32617"

# Reproject for accurate buffering
gta_boundary_m = (
    gpd.GeoSeries([gta_boundary], crs="EPSG:4326")
    .to_crs(CRS_METERS)
    .values[0]
)

# Outward buffer: +5000 m
gta_buffer_m = gta_boundary_m.buffer(BUFFER_DIST)

# Convert back to WGS84 so it matches centroid CRS
gta_boundary_expanded = (
    gpd.GeoSeries([gta_buffer_m], crs=CRS_METERS)
    .to_crs("EPSG:4326")
    .values[0]
)

# -------------------------------------------

EPS_THRESH = 0.0001  # treat counts <1 as null/no-data

for year in CENSUS_25:
    in_path = f"../data/language/{year}/num_speakers_centroid_{year}.gpkg"
    out_path = f"../data/language/{year}/num_speakers_centroid_{year}.json"

    print(f"Processing year {year}...")

    # Load centroids
    gdf = gpd.read_file(in_path).to_crs("EPSG:4326")

    # Coordinates
    gdf["x"] = gdf.geometry.x
    gdf["y"] = gdf.geometry.y

    # --- FILTER: keep only points within 5 km outside GTA ---
    gdf = gdf[gdf.geometry.within(gta_boundary_expanded)].copy()

    # Nullify tiny values
    num_cols = [c for c in gdf.columns if c.startswith("num_")]
    for col in num_cols:
        gdf[col] = gdf[col].where(gdf[col] >= EPS_THRESH, other=np.nan)

    # Prepare export DataFrame
    df = gdf[["x", "y"] + num_cols]

    # Export JSON
    df.to_json(out_path, orient="records", indent=2, force_ascii=False)

    print(f"✓ Saved {out_path} ({len(df)} rows)")


Processing year 1971...
✓ Saved ../data/language/1971/num_speakers_centroid_1971.json (2809 rows)
Processing year 1996...
✓ Saved ../data/language/1996/num_speakers_centroid_1996.json (2809 rows)
Processing year 2021...
✓ Saved ../data/language/2021/num_speakers_centroid_2021.json (2809 rows)
